In [1]:
import numpy as np
import pandas as pd
import random
import tqdm
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import seaborn as sns 
import yaml
from pathlib import Path

from util.functions import logistic
from structure_generation.path_connection_gen import ProceduralGraphGenerator, StatsUtils, GraphStructureGenerator




In [2]:

conf = yaml.safe_load(Path('config.yml').read_text())['reachability']


In [3]:
geometric_graph_conf = conf['random_geometric']


In [38]:
results_dict_list = []

for graph_rad in geometric_graph_conf['graph_edge_radii']:
    results_dict = {
        "timesteps_saturation" : [],
        "fraction_infected_list" : [],
        "info_dict" : []
    }
    print(f"simulation run for graph radius {graph_rad}")
    mean_degree_simulation_runs = []
    for i, repeat in tqdm.tqdm(enumerate(range(conf['num_simulation_runs']))):
        graphgen = GraphStructureGenerator(
            structure_name=conf['structure_name'], 
            num_nodes=int(geometric_graph_conf['nodes']), 
            graph_edge_radius = float(graph_rad)
        )
        graph = graphgen.initial_adj_matrix
        graph_rand = graphgen.get_graph_structure().initial_adj_matrix

        x = ProceduralGraphGenerator(graph)
        infection_matrix_list, timesteps_saturation, fraction_infected_list, info_dict = x.infect_till_saturation(
            modality="causal", verbose= False
        )
        results_dict["timesteps_saturation"].append(timesteps_saturation)
        results_dict["fraction_infected_list"].append(fraction_infected_list)
        if i == conf['num_simulation_runs'] - 1: 
            results_dict["info_dict"].append(info_dict)
    
    results_dict_list.append(results_dict)

simulation run for graph radius 0.025


5it [00:00, 40.41it/s]


simulation run for graph radius 0.05


5it [00:20,  4.15s/it]


In [32]:
results_dict_list[0].keys()

dict_keys(['timesteps_saturation', 'fraction_infected_list', 'info_dict'])

In [33]:
import itertools
for result in results_dict_list: 
    for key in result.keys():
        if isinstance(result[key], list): 
            if key == "fraction_infected_list":
                # Pad the list to ones to the longest saturation length, find the mean across all simulations and the std at each timestep
                padded_list = np.array(
                    list(zip(*itertools.zip_longest(*result[key], fillvalue=1)))
                )
                result[key] = np.mean(padded_list, axis=0)
                print(result[key])
            else: 
                pass

[0.18365801 0.29556277 0.43192641 0.61883117 0.69383117 0.88409091
 0.975      0.975      0.975      0.975      0.975      1.        ]
[0.00593191 0.00593191 0.00593191 ... 0.90694365 0.90694365 0.90694365]


In [37]:
result

{'timesteps_saturation': [1839, 1086, 2000, 2000, 2000],
 'fraction_infected_list': array([0.00593191, 0.00593191, 0.00593191, ..., 0.90694365, 0.90694365,
        0.90694365]),
 'info_dict': [{'average_degree': 4.574626865671642,
   'num_nodes': 268,
   'clustering_coefficient': 0.5450808726928131,
   'degree_assortivity': 0.6987979959753656,
   'mean_shortest_pathlength': 18.567834982391414}]}

In [ ]:
results_dict_list[0]

{'timesteps_saturation': [10, 11, 4, 14, 6],
 'fraction_infected_list': [[0.25,
   0.25,
   0.25,
   0.25,
   0.25,
   0.25,
   0.25,
   0.25,
   0.375,
   1.0],
  [0.2222222222222222,
   0.2222222222222222,
   0.3333333333333333,
   0.5555555555555556,
   0.5555555555555556,
   0.5555555555555556,
   0.5555555555555556,
   0.8888888888888888,
   0.8888888888888888,
   0.8888888888888888,
   1.0],
  [0.5714285714285714, 0.7142857142857143, 0.7142857142857143, 1.0],
  [0.1111111111111111,
   0.1111111111111111,
   0.1111111111111111,
   0.1111111111111111,
   0.1111111111111111,
   0.1111111111111111,
   0.1111111111111111,
   0.1111111111111111,
   0.1111111111111111,
   0.3333333333333333,
   0.8888888888888888,
   0.8888888888888888,
   0.8888888888888888,
   1.0],
  [0.4, 0.4, 0.4, 0.4, 0.6, 1.0]],
 'info_dict': [2.5, 2.2222222222222223, 2.0, 2.888888888888889, 3.4]}

In [ ]:
try: 
            #Fit a logistic curve to the simulated infection data for one simulation run, generate data with this logistic, use to find residuals in fit
            p, cov = curve_fit(logistic, timesteps, fraction_infected_list)
            logistic_curve_data = logistic(timesteps, *p)
            residuals_dict[graph_rad].extend(fraction_infected_list - logistic_curve_data)
            mean_degree_simulation_runs.append(average_degree)
        except RuntimeError as e:
            print(e)
            pass 
    average_degrees.append(np.mean(mean_degree_simulation_runs))

#Update keys of dictionary so that keys are the (averaged) mean degree of the simulations networks
residuals_dict = {k : v for k,v in list(zip(average_degrees, residuals_dict.values()))}
least_noisy_data = residuals_dict
for key in residuals_dict.keys(): 
    sns.kdeplot(residuals_dict[key], label=f"mean degree : {round(key, 2)}")
plt.legend()
plt.show()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 7)